In [1]:
import pandas as pd
import plotly.graph_objects as go
from collections import defaultdict

# Load the CSV file
file_path = "./solar_system_probe_data/5_Solar_System_Probes_with_Launches.csv"
satellite_data = pd.read_csv(file_path)

# Extract year from 'Launch Date' and 'Last Contact'
satellite_data["Launch Year"] = pd.to_numeric(satellite_data["Launch Date"], errors="coerce")
satellite_data["Last Contact Year"] = pd.to_numeric(satellite_data["Last Contact"], errors="coerce")

# Removing rows where launch year or last contact year is NaN
satellite_data.dropna(subset=["Launch Year", "Last Contact Year"], inplace=True)

# Initialize a dictionary to hold the count of active satellites per year per destination
active_satellites = defaultdict(lambda: defaultdict(int))

# Iterate through each row to count active satellites
for _, row in satellite_data.iterrows():
    launch_year = int(row["Launch Year"])
    last_contact_year = int(row["Last Contact Year"])
    destination = row["Destination"]

    # Counting active satellites for each year in the range from launch to last contact
    for year in range(launch_year, last_contact_year + 1):
        active_satellites[year][destination] += 1

# Convert the dictionary to a DataFrame suitable for Plotly
years = sorted(active_satellites.keys())
destinations = sorted(list(set().union(*[active_satellites[year].keys() for year in years])))

# Initialize an empty DataFrame with a list of destinations as columns
stacked_data = pd.DataFrame(index=years, columns=destinations).fillna(0)

# Populate the DataFrame
for year in years:
    for destination in destinations:
        stacked_data.loc[year, destination] = active_satellites[year].get(destination, 0)

# Creating the stacked bar chart
fig = go.Figure()
for destination in destinations:
    fig.add_trace(
        go.Bar(
            name=destination,
            x=stacked_data.index,
            y=stacked_data[destination],
        )
    )

# Update layout
fig.update_layout(
    barmode="stack",
    title="Active Satellites Over the Years by Destination",
    xaxis_title="Year",
    yaxis_title="Number of Active Satellites"
)

fig.show()
